In [3]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

# 自訂 function 可重新 reload
from importlib import reload
import sys # 到其他資料夾
sys.path.append('/home/hchuang/Documents/Project/SSSD_CP/src')
from utils import util_inference_result
reload(util_inference_result)
from utils.util_inference_result import *

from sklearn.metrics import mean_squared_error

# Set path

In [4]:
main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/results/ar1/'

# 設定 1  T=3
# sub_folder = 'ar1-6/T200_beta00.0001_betaT0.02/imputaiton_multiple_40k_test/' # M = 100
# sub_folder = 'ar1-6/T200_beta00.0001_betaT0.02/imputaiton_multiple_40k/' # M = 1000

# 設定 2  T=10
# sub_folder = 'ar1-4/T200_beta00.0001_betaT0.02/imputaiton_multiple_40k_test/' # M = 100
sub_folder = 'ar1-4/T200_beta00.0001_betaT0.02/imputaiton_multiple_40k/' # M = 1000


# 設定 2  T=100
# sub_folder = 'ar1-5/T200_beta00.0001_betaT0.02/imputaiton_multiple_10k_test/' # M = 100
sub_folder = 'ar1-5/T200_beta00.0001_betaT0.02/imputaiton_multiple_10k/' # M = 1000


# Read data and Calculate metrics

The metrics are shown in the thesis 4.4.1

In [ ]:
def metric_1(var1, var2):
    # output: metric 1 and its standard error
    var1 = var1.ravel()
    var2 = var2.ravel()
    stat = ((var1 - var2)**2)
    return round(mean_squared_error(var1, var2),5), round(np.std(stat),5)

def metric_2(var1):
    var1 = var1.ravel()
    stat = ((var1 - 1)**2)
    return round(np.mean(stat),5), round(np.std(stat),5)

def metric_3(var1):
    var1 = var1.ravel()
    stat = var1 - 1
    return round(np.mean(var1) - 1,5), round(np.std(stat),5)



In [ ]:
data = np.load(main_folder + sub_folder + 'original0.npy')

last_pred = np.expand_dims(np.empty(data.shape), axis = 0)

for i in range(1000):
    pred = np.load(main_folder + sub_folder + str(i) + '_imputation0.npy')
    pred = np.expand_dims(pred, axis = 0)
    last_pred = np.vstack((last_pred, pred))


last_pred = last_pred[1:(last_pred.shape[0]+1),:,:,:]
p = np.mean(last_pred[:,:,:,-1], axis = 0)   
v = np.var(last_pred[:,:,:,-1], axis = 0)

a = p
b = data[:,:,-2].reshape(-1) # y_{T-1}
c = data[:,:,-1].reshape(-1) # y_{T}

print(f'metric 1 = MSE of Exp = {metric_1(a, b*0.8)}')
print(f'metric 2 = MSE of Var = {metric_2(v)}')
print(f'metric 3 = Bias of Var = {metric_3(v)}')

metric 1 = E[(p-0.8*y_T-1)^2] = (0.24385, 0.3371)
metric 2 = E[var(\hat(y))] = (-0.95221, 0.01803)
